<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/03_01_char_rnn_middle_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Nov 22 14:26 .
drwxr-xr-x 1 root root 4096 Nov 22 14:19 ..
drwxr-xr-x 1 root root 4096 Nov 20 16:17 .config
drwx------ 4 root root 4096 Nov 22 14:26 drive
drwxr-xr-x 1 root root 4096 Nov 15 16:31 sample_data


In [4]:
%cd drive/My Drive/dataset

/content/drive/My Drive/dataset


### Sing a Song of Sixpence

Load Text

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [6]:
# load text
raw_text = load_doc('rhyme.txt')
print(raw_text)

Sing a song of sixpence,
A pocket full of rye.
Four and twenty blackbirds,
Baked in a pie.
 
When the pie was opened
The birds began to sing;
Wasn't that a dainty dish,
To set before the king.
 
The king was in his counting house,
Counting out his money;
The queen was in the parlour,
Eating bread and honey.
 
The maid was in the garden,
Hanging out the clothes,
When down came a blackbird
And pecked off her nose.



Clean Text

In [7]:
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

Sing a song of sixpence, A pocket full of rye. Four and twenty blackbirds, Baked in a pie. When the pie was opened The birds began to sing; Wasn't that a dainty dish, To set before the king. The king was in his counting house, Counting out his money; The queen was in the parlour, Eating bread and honey. The maid was in the garden, Hanging out the clothes, When down came a blackbird And pecked off her nose.


Create Sequences

In [8]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
	# select sequence of tokens
	seq = raw_text[i-length:i+1]
	# store
	sequences.append(seq)
print('Total Sequences: %d' % len(sequences))
print(sequences[0])
print(sequences[1])
print(sequences[397])
print(sequences[398])

Total Sequences: 399
Sing a song
ing a song 
ff her nose
f her nose.


Save Sequences

In [0]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [0]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

Complete Example

In [11]:
# load text
char_text = load_doc('char_sequences.txt')
print(char_text)

Sing a song
ing a song 
ng a song o
g a song of
 a song of 
a song of s
 song of si
song of six
ong of sixp
ng of sixpe
g of sixpen
 of sixpenc
of sixpence
f sixpence,
 sixpence, 
sixpence, A
ixpence, A 
xpence, A p
pence, A po
ence, A poc
nce, A pock
ce, A pocke
e, A pocket
, A pocket 
 A pocket f
A pocket fu
 pocket ful
pocket full
ocket full 
cket full o
ket full of
et full of 
t full of r
 full of ry
full of rye
ull of rye.
ll of rye. 
l of rye. F
 of rye. Fo
of rye. Fou
f rye. Four
 rye. Four 
rye. Four a
ye. Four an
e. Four and
. Four and 
 Four and t
Four and tw
our and twe
ur and twen
r and twent
 and twenty
and twenty 
nd twenty b
d twenty bl
 twenty bla
twenty blac
wenty black
enty blackb
nty blackbi
ty blackbir
y blackbird
 blackbirds
blackbirds,
lackbirds, 
ackbirds, B
ckbirds, Ba
kbirds, Bak
birds, Bake
irds, Baked
rds, Baked 
ds, Baked i
s, Baked in
, Baked in 
 Baked in a
Baked in a 
aked in a p
ked in a pi
ed in a pie
d in a pie.
 in a pie. 
in a pie. W
n a pie. Wh
 a p

### Train Language Model

Load Data

In [0]:
from numpy import array
from pickle import dump
import pickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils  import to_categorical

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [14]:
# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
#print (raw_text)

lines = raw_text.split('\n')
print (type(lines))
print (lines)

<class 'list'>
['Sing a song', 'ing a song ', 'ng a song o', 'g a song of', ' a song of ', 'a song of s', ' song of si', 'song of six', 'ong of sixp', 'ng of sixpe', 'g of sixpen', ' of sixpenc', 'of sixpence', 'f sixpence,', ' sixpence, ', 'sixpence, A', 'ixpence, A ', 'xpence, A p', 'pence, A po', 'ence, A poc', 'nce, A pock', 'ce, A pocke', 'e, A pocket', ', A pocket ', ' A pocket f', 'A pocket fu', ' pocket ful', 'pocket full', 'ocket full ', 'cket full o', 'ket full of', 'et full of ', 't full of r', ' full of ry', 'full of rye', 'ull of rye.', 'll of rye. ', 'l of rye. F', ' of rye. Fo', 'of rye. Fou', 'f rye. Four', ' rye. Four ', 'rye. Four a', 'ye. Four an', 'e. Four and', '. Four and ', ' Four and t', 'Four and tw', 'our and twe', 'ur and twen', 'r and twent', ' and twenty', 'and twenty ', 'nd twenty b', 'd twenty bl', ' twenty bla', 'twenty blac', 'wenty black', 'enty blackb', 'nty blackbi', 'ty blackbir', 'y blackbird', ' blackbirds', 'blackbirds,', 'lackbirds, ', 'ackbirds

In [15]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
print (chars)

mapping = dict((c, i) for i, c in enumerate(chars))
print (mapping)

['\n', ' ', "'", ',', '.', ';', 'A', 'B', 'C', 'E', 'F', 'H', 'S', 'T', 'W', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'x', 'y']
{'\n': 0, ' ': 1, "'": 2, ',': 3, '.': 4, ';': 5, 'A': 6, 'B': 7, 'C': 8, 'E': 9, 'F': 10, 'H': 11, 'S': 12, 'T': 13, 'W': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'q': 30, 'r': 31, 's': 32, 't': 33, 'u': 34, 'w': 35, 'x': 36, 'y': 37}


In [30]:
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)

print ((type(sequences))) 
print (sequences[0])    # int sequence
print (sequences[397])
print (sequences[398])

<class 'list'>
[12, 23, 27, 21, 1, 15, 1, 32, 28, 27, 21]
[20, 20, 1, 22, 19, 31, 1, 27, 28, 32, 19]
[20, 1, 22, 19, 31, 1, 27, 28, 32, 19, 4]


In [17]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 38


Split Input and Output

In [18]:
# separate into input and output
sequences = array(sequences)
print (sequences.shape)

X, y = sequences[:,:-1], sequences[:,-1]
print (X.shape)
print (y.shape)

(399, 11)
(399, 10)
(399,)


In [19]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
print (sequences[0].shape)   # (10,38)
print (sequences[398].shape) # (10,38)

X = array(sequences)
print (X.shape) # (399, 10, 38)

print (y.shape) # (399,)
y = to_categorical(y, num_classes=vocab_size)
print (y.shape) # (399, 38)
print (y[0])  # g -> 21th one-hot

(10, 38)
(10, 38)
(399, 10, 38)
(399,)
(399, 38)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
print (X.shape[1])
print (X.shape[2])

10
38


In [21]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 75)                34200     
_________________________________________________________________
dense (Dense)                (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
print (X.shape) # (399, 10, 38)
print (y.shape) # (399, 38)
model.fit(X, y, epochs=100, verbose=2)

(399, 10, 38)
(399, 38)
Train on 399 samples
Epoch 1/100
399/399 - 3s - loss: 3.6088 - accuracy: 0.0977
Epoch 2/100
399/399 - 0s - loss: 3.4952 - accuracy: 0.1880
Epoch 3/100
399/399 - 0s - loss: 3.1796 - accuracy: 0.1905
Epoch 4/100
399/399 - 0s - loss: 3.0541 - accuracy: 0.1905
Epoch 5/100
399/399 - 0s - loss: 3.0054 - accuracy: 0.1905
Epoch 6/100
399/399 - 0s - loss: 2.9923 - accuracy: 0.1905
Epoch 7/100
399/399 - 0s - loss: 2.9720 - accuracy: 0.1905
Epoch 8/100
399/399 - 0s - loss: 2.9629 - accuracy: 0.1905
Epoch 9/100
399/399 - 0s - loss: 2.9498 - accuracy: 0.1905
Epoch 10/100
399/399 - 0s - loss: 2.9349 - accuracy: 0.1905
Epoch 11/100
399/399 - 0s - loss: 2.9178 - accuracy: 0.1905
Epoch 12/100
399/399 - 0s - loss: 2.9009 - accuracy: 0.1930
Epoch 13/100
399/399 - 0s - loss: 2.8756 - accuracy: 0.1905
Epoch 14/100
399/399 - 0s - loss: 2.8618 - accuracy: 0.1905
Epoch 15/100
399/399 - 0s - loss: 2.8451 - accuracy: 0.2431
Epoch 16/100
399/399 - 0s - loss: 2.8144 - accuracy: 0.1905
Epoc

In [0]:
# save the model to file
model.save('model.h5')

In [0]:
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

### Generate Text

Load Model

In [0]:
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [0]:
# load the model
model = tf.keras.models.load_model('model.h5')

In [0]:
# load the mapping
mapping = pickle.load(open('mapping.pkl', 'rb'))

In [29]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

Sing a song of sixpence, A poc
king was in his counting house
hello worlke Fesk Tof e effl o
